## Draft

In [1]:
from pymongo import MongoClient
import pprint

import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# Requests sends and recieves HTTP requests.
import requests

# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup

import json
import time
import copy

In [2]:
from nba_api.stats.endpoints import commonplayerinfo

### General Dataset Information

Question: Is offensive or defensive play more effective in achieving a win?

Dataset: using proxies, such as either offensive (ORtg) and defensive (DRtg) ratings or offense and defense four factors, for efficient offensive and defensive play respectively, which contributes the most to achieving a win?

I’d like to look at nba game stats to understand whether there is a difference between good offensive play and good defensive play for game wins. 

### Dataset Preparation


### Data Analysis

In [4]:
from nba_api.stats.endpoints import playercareerstats
# Anthony Davis
career = playercareerstats.PlayerCareerStats(player_id='203076')
career.get_data_frames()[0]

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,203076,2012-13,00,1610612740,NOH,20.0,64,60,1846.0,349,...,0.751,165,357,522,63,75,112,89,158,867
1,203076,2013-14,00,1610612740,NOP,21.0,67,66,2358.0,522,...,0.791,207,466,673,105,89,189,109,200,1394
2,203076,2014-15,00,1610612740,NOP,22.0,68,68,2455.0,642,...,0.805,173,523,696,149,100,200,95,141,1656
3,203076,2015-16,00,1610612740,NOP,23.0,61,61,2164.0,560,...,0.758,130,497,627,116,78,125,121,148,1481
4,203076,2016-17,00,1610612740,NOP,24.0,75,75,2708.0,770,...,0.802,172,712,884,157,94,167,181,168,2099
5,203076,2017-18,00,1610612740,NOP,25.0,75,75,2705.0,772,...,0.827,185,641,826,173,112,192,160,154,2091
6,203076,2018-19,00,1610612740,NOP,26.0,56,56,1850.0,530,...,0.794,174,498,672,218,88,135,112,132,1452
7,203076,2019-20,00,1610612747,LAL,27.0,62,62,2131.0,551,...,0.846,142,435,577,200,91,143,154,156,1618
8,203076,2020-21,00,1610612747,LAL,27.0,23,23,755.0,204,...,0.715,47,146,193,70,31,41,45,42,518


### Hypothesis Tests

### Conclusion

#### How will my project be evaluated?
We’ve put together some project assessment criteria. Read over them carefully before you get to work. Here’s what project reviewers will look at when assessing your project:

How you explain the problems identified in the data
How you prepare the data for analysis
What graphs you plot for distributions
How you interpret the resulting graphs
How you calculate the standard deviation and variance
Whether you formulate the alternative and null hypotheses
What methods you use to test hypotheses
Whether you interpret the results of your hypothesis tests
Whether you follow the project structure and keep the code tidy
The conclusions you come to
Whether you leave comments at each step